In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

dtype = torch.FloatTensor

In [2]:
#准备数据，这里是一个句子
sentence = (
    'GitHub Actions makes it easy to automate all your software workflows '
    'from continuous integration and delivery to issue triage and more'
)
#制作词典
word2idx = {w: i for i, w in enumerate(list(set(sentence.split())))}
idx2word = {i: w for i, w in enumerate(list(set(sentence.split())))}
n_class = len(word2idx) # 多分类问题 每个词都是一个特殊的类
max_len = len(sentence.split())
n_hidden = 5

下一步是数据填充。
对于下面的循环来讲，input 的第一个赋值语句会将第一个词 Github 对应的索引存起来。input 的第二个赋值语句会将剩下的 max_len - len(input) 都用 0 去填充
target 就会变成 Actions

比如 i=0时

GitHub 0 0 0 0 0 0 0 0 0 0

然后我们一个一个去预测下一个单词

GitHub Actions 0 0 0 0 0 0 0 0 0

input 的长度永远保持 max_len(=21)，并且循环了 max_len-1 次，所以最终 input_batch 的维度是 [max_len - 1, max_len, n_class]

In [4]:
def make_data(sentence):
    input_batch = []
    target_batch = []

    words = sentence.split()
    for i in range(max_len - 1):#循环20次
        input = [word2idx[n] for n in words[:(i + 1)]]#[1]
        input = input + [0] * (max_len - len(input))#[1] + [0] * 20 这里0起到占位符的作用
        target = word2idx[words[i + 1]]#预测的字符 [2]
        input_batch.append(np.eye(n_class)[input]) # np.eye()的函数，除了生成对角阵外，还可以将一个label数组，
        #大小为(1,m)或者(m,1)的数组，转化成one-hot数组。
        target_batch.append(target)

    return torch.Tensor(input_batch), torch.LongTensor(target_batch)

# input_batch: [max_len - 1, max_len, n_class]
input_batch, target_batch = make_data(sentence)
dataset = Data.TensorDataset(input_batch, target_batch)
loader = Data.DataLoader(dataset, 16, True)

In [5]:
class BiLSTM(nn.Module):
    def __init__(self):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, bidirectional=True)
        # fc
        self.fc = nn.Linear(n_hidden * 2, n_class)

    def forward(self, X):
        # X: [batch_size, max_len, n_class]
        batch_size = X.shape[0]
        input = X.transpose(0, 1)  #  这里交换的话是必须把batch_size放在第二个维度上 input : [max_len, batch_size, n_class]

        hidden_state = torch.randn(1*2, batch_size, n_hidden)   # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
        cell_state = torch.randn(1*2, batch_size, n_hidden)     # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]

        outputs, (_, _) = self.lstm(input, (hidden_state, cell_state))
        outputs = outputs[-1]  # [batch_size, n_hidden * 2]
        model = self.fc(outputs)  # model : [batch_size, n_class]
        return model

model = BiLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
# Training
for epoch in range(10000):
    for x, y in loader:
      pred = model(x)
      loss = criterion(pred, y)
      if (epoch + 1) % 1000 == 0:
          print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

# Pred
predict = model(input_batch).data.max(1, keepdim=True)[1]
print(sentence)
print([idx2word[n.item()] for n in predict.squeeze()])

Epoch: 1000 cost = 1.598237
Epoch: 1000 cost = 1.769609
Epoch: 2000 cost = 1.308757
Epoch: 2000 cost = 1.373865
Epoch: 3000 cost = 0.947228
Epoch: 3000 cost = 1.275564
Epoch: 4000 cost = 0.881621
Epoch: 4000 cost = 0.577002
Epoch: 5000 cost = 0.610097
Epoch: 5000 cost = 1.088931
Epoch: 6000 cost = 0.558699
Epoch: 6000 cost = 0.733245
Epoch: 7000 cost = 0.433467
Epoch: 7000 cost = 0.843847
Epoch: 8000 cost = 0.502204
Epoch: 8000 cost = 0.230651
Epoch: 9000 cost = 0.354675
Epoch: 9000 cost = 0.649028
Epoch: 10000 cost = 0.379112
Epoch: 10000 cost = 0.403667
GitHub Actions makes it easy to automate all your software workflows from continuous integration and delivery to issue triage and more
['Actions', 'makes', 'it', 'easy', 'to', 'automate', 'all', 'your', 'workflows', 'workflows', 'from', 'integration', 'integration', 'and', 'and', 'to', 'issue', 'triage', 'and', 'and']
